# Сборный проект "Исследования уровня потребительской лояльности пользователей телекоммуникационной компании"

# ВВЕДЕНИЕ

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 

## Цели и задачи

Цели исследования - ответить на вопросы:

- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых?
- Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- 
Какой общий NPS среди всех опрошеннх ?- 
Как можно описать клиентов, которые относятся к группе cторонников (англ. promotsже):

## Подготовка данных

Импортируем необходимые библиотеки для обработки данных.

In [4]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

Подготовим необходимые запросы и выгрузим целевой датасет. 

In [8]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = 'C:/Users/sutul/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [9]:
query = """
SELECT u.user_id,
        u.lt_day,
            CASE
                WHEN u.lt_day < 365 THEN 'TRUE'
                ELSE 'FALSE'
            END is_new,
        u.age,
            CASE
                WHEN u.gender_segment = 1 THEN 'женщина'
                WHEN u.gender_segment = 0 THEN 'мужчина'
                ELSE 'неизвестно'
            END gender_segment,
        u.os_name,
        u.cpe_type_name,
        l.country,
        l.city,
        ags.title AS age_segment,
        ts.title AS traffic_segment,
        ls.title AS lifetime_segment,
        u.nps_score,
            CASE
                WHEN u.nps_score <= 6 THEN 'критики'
                WHEN u.nps_score >= 9 THEN 'сторонники'
                ELSE 'нейтралы'
            END nps_group
FROM user AS u
JOIN location AS l ON u.location_id = l.location_id
JOIN age_segment AS ags ON u.age_gr_id = ags.age_gr_id
JOIN traffic_segment AS ts ON u.tr_gr_id = ts.tr_gr_id
JOIN lifetime_segment AS ls ON u.lt_gr_id = ls.lt_gr_id;

"""

In [10]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,FALSE,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонники
1,A001WF,2344,FALSE,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонники
2,A003Q7,467,FALSE,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонники


In [11]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


# МОЙ ДАШБОРД С ПРЕЗЕНТАЦИЕЙ МОЖНО ПОСМОТРЕТЬ [ЗДЕСЬ](https://public.tableau.com/app/profile/lev.sutulov/viz/NPS_analysis_16877870962290/sheet0?publish=yes)